<a href="https://colab.research.google.com/github/Bruno-yano/Atividades-IAMA/blob/main/Soja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pathlib


In [35]:
# prompt: please generate code to load a folder in my google drive

from google.colab import drive
drive.mount('/content/drive')

import os
path = "/content/drive/My Drive/Base_de_dados/Soja" # replace with your folder path
pathlib.Path(path)

image_count = len(list(pathlib.Path(path).glob('*/*.jpg')))
print(image_count)

#define some parameters to the loader
batch_size = 32
img_height = 227
img_width = 227

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
5513
